In [4]:
from neo4j import GraphDatabase
import pandas as pd

# Configura tus credenciales y URL de conexión
URI = "bolt://localhost:7687" 
USER = "neo4j"
PASSWORD = "password"

# Crear driver
driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))

In [11]:
node_properties_query = """
MATCH (n)
UNWIND keys(n) AS prop
RETURN DISTINCT prop AS property_name
"""

with driver.session() as session:
    result = session.run(node_properties_query)
    columns = [record["property_name"] for record in result]

print(columns)


['doi', 'author_count', 'publication_date', 'abstract', 'title', 'scopus_id', 'neo4jImportId', 'affiliation_count', 'pk', 'name', 'country', 'city', 'auth_name', 'citation_count', 'initials', 'current_affiliation', 'first_name', 'last_name', 'updated', 'cursor', 'next_url']


In [5]:
def export_articles_to_csv():
    query = """
    MATCH (a)
    WHERE a.scopus_id IS NOT NULL
      AND a.title IS NOT NULL
      AND a.abstract IS NOT NULL
    RETURN
      a.scopus_id as scopus_id,
      a.title as title,
      a.abstract as abstract
    """

    # Ejecutar la consulta dentro de una sesión
    with driver.session() as session:
        result = session.run(query)
        df = pd.DataFrame([
            {
                'scopus_id': record['scopus_id'],
                'title': record['title'],
                'abstract': record['abstract']
            }
            for record in result
        ])
    
    # Guardar en CSV
    df.to_csv("articles_scopus.csv", index=False, encoding="utf-8")
    print("✅ Exportación completada: articles_scopus.csv")

if __name__ == "__main__":
    export_articles_to_csv()


✅ Exportación completada: articles_scopus.csv
